In [1]:
# Python 2/3 compatibility.
from __future__ import print_function
import sys
import os
import io
sys.path.append('../')
sys.path.append('../../')
import pprint
import time
import threading
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import interpolate
import PIL
from Xlib import X, display, Xutil
from Xlib.ext import randr, xinerama
import subprocess
import datetime
import pandas as pd
os.system('sh -c \'echo 2000 > /sys/module/usbcore/parameters/usbfs_memory_mb\'')
import warnings
warnings.filterwarnings('ignore')

import _thread

import PySpin

from ipywidgets import interact, interactive, fixed, interact_manual, Video
import ipywidgets as widgets
import IPython.display as ipd
from ipyfilechooser import FileChooser
import ipysheet


from camera import Camera
from illumination import Illumination
from stage import Stage
from widgetCtr import WidgetsPanel

import pyfirmata
import pause

import thorlabs_apt_device



# Setup connections

## List cameras

In [2]:
# camera = Camera(camNo = 0, width = 5000, height = 3000, binning = 2, \
#                 exposure = 90000, framerate = 10, \
#                 pixelFormat = 16, gain = 5)
# camera.setup_single_camera(isPrint = True)
# time.sleep(2)

In [3]:
try:
    test_file = open('test.txt', 'w+')
except IOError:
    print('Unable to write to current directory. Please check permissions.')
    input('Press Enter to exit...')
    exit(0)

test_file.close()
os.remove(test_file.name)

# Retrieve singleton reference to system object
system = PySpin.System.GetInstance()

# Get current library version
version = system.GetLibraryVersion()
print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

# Retrieve list of cameras from the system
cam_list = system.GetCameras() # Somehow cannot be done within Camera class.

num_cameras = cam_list.GetSize()

print('Number of cameras detected: %d' % num_cameras)

# Finish if there are no cameras
if num_cameras == 0:

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    print('Not enough cameras!')
    input('Done! Press Enter to exit...')
    exit(0)

# Get camera
for _, cam in enumerate(cam_list):
    1
    
camera = Camera(cam, width = 5000, height = 3000, binning = 2, \
                exposure = 90000, framerate = 10, \
                pixelFormat = 16, gain = 5)
camera.setup_single_camera(isPrint = True)
# camera.cam.EndAcquisition()
time.sleep(2)

Library version: 2.5.0.80
Number of cameras detected: 1

*** DEVICE INFORMATION ***

DeviceID: USB\VID_1E10&SRL_010AE589\&2&5&2
DeviceSerialNumber: 17491337
DeviceVendorName: Point Grey Research
DeviceModelName: Grasshopper3 GS3-U3-89S6M
DeviceType: USB3Vision
DeviceDisplayName: Point Grey Research
DeviceAccessStatus: OpenReadWrite
DeviceVersion: FW:v2.25.3.01 FPGA:v2.02
DeviceDriverVersion: none : 0.0.0.0
DeviceUserID: 
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&SRL_010AE589\&2&5&2
DeviceLocation: 
DeviceCurrentSpeed: SuperSpeed
GUIXMLLocation: Device
GUIXMLPath: Input.xml
GenICamXMLLocation: Device
GenICamXMLPath: 
DeviceU3VProtocol: 1
DevicePortId: 
2048 1080
Image Shape: (1080, 2048)


## Generate illumination object

In [4]:
#os.system('xrandr --display :0.0 --size 640x480')
# illumination = [Illumination(display=display.Display(':0.0'), screen_id = 0, lht=10)]

illumination = [Illumination(display=display.Display(':0.0'), screen_id = 0, lht=10),
               Illumination(display=display.Display(':0.0'), screen_id = 1, lht=10)]

## Generate stage object

In [5]:
stage = Stage()

In [6]:
widgetPanel = WidgetsPanel(camera, illumination, stage, r'/media/pLSM/Samsung_T5/nmeth/')

Received unknown event notification from APT device 33.
Message mot_get_kcubekstloopparams has unrecognised channel=2.
Message mot_get_kcubekstloopparams has unrecognised channel=2.


## Start display and configuration

In [7]:
%matplotlib inline

ipd.display(widgetPanel.outputPanel)


In [8]:
# widgetPanel.save_metadata_file(r'.')

# Debugging

In [9]:
# stage.stop_all()